<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
rm -rf /geological-image-similarity

**Importar arquivos de configuração do Kaglle**

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [4]:
!kaggle datasets download tanyadayanand/geological-image-similarity
! unzip -qq geological-image-similarity

 89% 30.0M/33.6M [00:00<00:00, 80.1MB/s]
100% 33.6M/33.6M [00:00<00:00, 95.7MB/s]


### Data Preprocessing

In [5]:
andesite_dir = glob.glob('geological_similarity/andesite/*.jpg');gneiss_dir = glob.glob('geological_similarity/gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


5000
5000
4998
5000
5000


In [ ]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [12]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']
class_names[0]

'andesite'

In [16]:
for i in andesite_dir[0:500]:
    andesite_dir.append([i,class_names[0]])
for j in gneiss_dir[0:500]:
   gneiss_dir.append([j,class_names[1]])
df = andesite_dir + gneiss_dir
random.shuffle(df)
df

['geological_similarity/andesite/QC8I1.jpg',
 'geological_similarity/andesite/B1THF.jpg',
 'geological_similarity/andesite/JR6K0.jpg',
 'geological_similarity/gneiss/9ROFV.jpg',
 'geological_similarity/gneiss/P3FFR.jpg',
 'geological_similarity/gneiss/LJ5BA.jpg',
 ['geological_similarity/gneiss/53UPS.jpg', 'class_names[1]'],
 'geological_similarity/gneiss/AJNWW.jpg',
 'geological_similarity/andesite/DSZFL.jpg',
 'geological_similarity/andesite/ZNLL9.jpg',
 ['geological_similarity/andesite/IG42W.jpg', 'andesite'],
 'geological_similarity/gneiss/ATHY0.jpg',
 'geological_similarity/gneiss/RYK2T.jpg',
 ['geological_similarity/gneiss/TBNIY.jpg', 'class_names[1]'],
 'geological_similarity/gneiss/8VTXY.jpg',
 ['geological_similarity/andesite/LDEFO.jpg', 'print(class_names[0])'],
 'geological_similarity/gneiss/JX06P.jpg',
 'geological_similarity/gneiss/CEGRF.jpg',
 'geological_similarity/gneiss/A14EM.jpg',
 ['geological_similarity/gneiss/OUOHG.jpg', 'gneiss'],
 'geological_similarity/gneiss/N0

In [17]:
data_df = pd.DataFrame(df,columns=['PATH','LABEL'])
data_df

ValueError: ignored